# Sharing demos with others

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [ ]:
# Install required libraries for Transformers, datasets, evaluation, and Gradio interface
!uv pip install datasets evaluate transformers[sentencepiece]
!uv pip install gradio

In [ ]:
# Enhanced interface with custom title, description, and examples
# This demonstrates how to create professional-looking interfaces with rich metadata
title = "Ask Rick a Question"
description = """
The bot was trained to answer questions based on Rick and Morty dialogues. Ask Rick anything!
<img src="https://huggingface.co/spaces/course-demos/Rick_and_Morty_QA/resolve/main/rick.png" width=200px>
"""

# Article section provides additional context or links for users
article = "Check out [the original Rick and Morty Bot](https://huggingface.co/spaces/kingabzpro/Rick_and_Morty_Bot) that this demo is based off of."

# Create interface with professional presentation
gr.Interface(
    fn=predict,  # Function defined in previous cell
    inputs="textbox",  # Text input for questions
    outputs="text",    # Text output for answers
    title=title,                # Main title displayed at top
    description=description,    # HTML description with formatting and images
    article=article,           # Additional information at bottom
    examples=[["What are you doing?"], ["Where should we time travel to?"]],  # Sample inputs
).launch()

In [ ]:
# Public sharing with share=True parameter
# This creates a temporary public URL that others can access for 72 hours
# Very useful for demonstrations, presentations, or sharing with remote collaborators
gr.Interface(classify_image, "image", "label").launch(share=True)

In [ ]:
# Custom sketch recognition model - demonstrates loading and using a custom trained model
# This example shows how to integrate your own PyTorch models with Gradio
from pathlib import Path
import torch
import gradio as gr
from torch import nn

# Load class labels from file - these are the categories the model can recognize
LABELS = Path("class_names.txt").read_text().splitlines()

# Define the neural network architecture
# This CNN is designed for sketch/drawing recognition
model = nn.Sequential(
    # First convolutional block: 1 input channel (grayscale), 32 output channels
    nn.Conv2d(1, 32, 3, padding="same"),  # 3x3 convolution with same padding
    nn.ReLU(),                             # ReLU activation
    nn.MaxPool2d(2),                       # 2x2 max pooling (downsampling)
    
    # Second convolutional block: 32->64 channels
    nn.Conv2d(32, 64, 3, padding="same"),
    nn.ReLU(),
    nn.MaxPool2d(2),
    
    # Third convolutional block: 64->128 channels
    nn.Conv2d(64, 128, 3, padding="same"),
    nn.ReLU(),
    nn.MaxPool2d(2),
    
    # Flatten for fully connected layers
    nn.Flatten(),
    # Fully connected layers for classification
    nn.Linear(1152, 256),     # 1152 comes from flattened conv output size
    nn.ReLU(),
    nn.Linear(256, len(LABELS)),  # Output layer matches number of classes
)

# Load pre-trained weights from file
# map_location="cpu" ensures compatibility regardless of training device
state_dict = torch.load("pytorch_model.bin", map_location="cpu")
model.load_state_dict(state_dict, strict=False)
model.eval()  # Set to evaluation mode (disables dropout, batch norm training)


def predict(im):
    # Preprocess the input image for the model
    # Convert to tensor, add batch and channel dimensions, normalize to [0,1]
    x = torch.tensor(im, dtype=torch.float32).unsqueeze(0).unsqueeze(0) / 255.0
    
    # Run inference without gradient computation (saves memory and computation)
    with torch.no_grad():
        out = model(x)
    
    # Convert raw logits to probabilities using softmax
    probabilities = torch.nn.functional.softmax(out[0], dim=0)
    
    # Get top 5 predictions with their confidence scores
    values, indices = torch.topk(probabilities, 5)
    
    # Return as dictionary mapping class names to confidence scores
    return {LABELS[i]: v.item() for i, v in zip(indices, values)}

In [ ]:
# Real-time sketch recognition interface with advanced features
# Demonstrates live prediction and professional interface design
interface = gr.Interface(
    predict,
    inputs="sketchpad",    # Special input type for drawing/sketching
    outputs="label",       # Label output shows top predictions with confidence scores
    theme="huggingface",   # Use Hugging Face's branded theme
    title="Sketch Recognition",
    description="Who wants to play Pictionary? Draw a common object like a shovel or a laptop, and the algorithm will guess in real time!",
    article="<p style='text-align: center'>Sketch Recognition | Demo Model</p>",
    live=True,            # Enable live prediction: updates as user draws
)
interface.launch(share=True)  # Launch with public sharing enabled